In [0]:
# Mount ADLS Gen2
# Required each time the cluster is restarted which should be only on the first notebook as they run in order

tiers = ["bronze", "silver", "gold"]
adls_paths = {tier: f"abfss://{tier}@dataproject1.dfs.core.windows.net/" for tier in tiers}

# Accessing paths
bronze_adls = adls_paths["bronze"]
silver_adls = adls_paths["silver"]
gold_adls = adls_paths["gold"] 

dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)
     

[FileInfo(path='abfss://gold@dataproject1.dfs.core.windows.net/earthquake_events_gold/', name='earthquake_events_gold/', size=0, modificationTime=1743608111000)]

In [0]:
import json
from datetime import date, timedelta
     

In [0]:
# Get base parameters
dbutils.widgets.text("start_date", "")
dbutils.widgets.text("end_date", "")

start_date = dbutils.widgets.get("start_date")
end_date = dbutils.widgets.get("end_date")

In [0]:
# Construct the API URL with start and end dates provided by Data Factory, formatted for geojson output.
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={start_date}&endtime={end_date}"

In [0]:
url

'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=&endtime='

In [0]:
%python
%pip install requests

import requests
import json

try:
    # Make the GET request to fetch data
    response = requests.get(url)

    # Check if the request was successful
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    data = response.json().get('features', [])

    if not data:
        print("No data returned for the specified date range.")
    else:
        # Specify the ADLS path
        file_path = f"{bronze_adls}/{start_date}_earthquake_data.json"

        # Save the JSON data
        json_data = json.dumps(data, indent=4)
        dbutils.fs.put(file_path, json_data, overwrite=True)
        print(f"Data successfully saved to {file_path}")
except requests.exceptions.RequestException as e:
    print(f"Error fetching data from API: {e}")

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Wrote 11926936 bytes.
Data successfully saved to abfss://bronze@dataproject1.dfs.core.windows.net//_earthquake_data.json


In [0]:
data[0]

{'type': 'Feature',
 'properties': {'mag': 0.85,
  'place': '8 km NNW of The Geysers, CA',
  'time': 1743636533300,
  'updated': 1743636628109,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/nc75158967',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc75158967&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 11,
  'net': 'nc',
  'code': '75158967',
  'ids': ',nc75158967,',
  'sources': ',nc,',
  'types': ',nearby-cities,origin,phase-data,',
  'nst': 5,
  'dmin': 0.01798,
  'rms': 0.02,
  'gap': 91,
  'magType': 'md',
  'type': 'earthquake',
  'title': 'M 0.9 - 8 km NNW of The Geysers, CA'},
 'geometry': {'type': 'Point',
  'coordinates': [-122.811332702637, 38.8310012817383, 1.86000001430511]},
 'id': 'nc75158967'}

In [0]:
# Define your variables
output_data = {
    "start_date": start_date,
    "end_date": end_date,
    "bronze_adls": bronze_adls,
    "silver_adls": silver_adls,
    "gold_adls": gold_adls
}

# Serialize the dictionary to a JSON string
output_json = json.dumps(output_data)

# Log the serialized JSON for debugging
print(f"Serialized JSON: {output_json}")

# Return the JSON string
dbutils.notebook.exit(output_json)